# setup

In [1]:
import torch
import os

In [2]:
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

In [3]:
from dataloader import Cifar100
from runners.baseline import Runner

# init

In [4]:
model_label = 'resnet18'
model = torch.hub.load('pytorch/vision:v0.10.0', model_label, pretrained = False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 10
BATCH_SIZE = 128
NUM_WORKERS = 6

Using cache found in /home/josegfer/.cache/torch/hub/pytorch_vision_v0.10.0


In [5]:
trn_ds = Cifar100(split = 'train')
trn_loader = torch.utils.data.DataLoader(trn_ds, batch_size = BATCH_SIZE,
                                          shuffle = True, num_workers = NUM_WORKERS)

val_ds = Cifar100(split = 'val')
val_loader = torch.utils.data.DataLoader(val_ds, batch_size = BATCH_SIZE,
                                         shuffle = False, num_workers = NUM_WORKERS)

In [6]:
runner = Runner(device, model, model_label)

# train

In [7]:
runner.train(EPOCHS, trn_loader, val_loader)

-- epoch 0


  0%|          | 0/391 [00:00<?, ?it/s]/home/josegfer/miniconda3/envs/mirror/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 79/79 [00:11<00:00,  6.68it/s]


-- epoch 1


100%|██████████| 79/79 [00:11<00:00,  7.08it/s]


-- epoch 2


...

-- epoch 8


100%|██████████| 79/79 [00:11<00:00,  6.90it/s]


-- epoch 9


100%|██████████| 79/79 [00:11<00:00,  6.82it/s]


exporting model


# eval

In [8]:
acc = runner.acc(val_loader)
acc['acc']

  0%|          | 0/79 [00:00<?, ?it/s]

100%|██████████| 79/79 [00:11<00:00,  7.03it/s]


0.5444